<a href="https://colab.research.google.com/github/vlad-danaila/machine-learning-workout/blob/master/Exercise_10_Style_Transfer_Higher_Resolution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Style Transfer**

In [65]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [66]:
!pip install torch torchvision

In [0]:
import numpy as np
import torch as t
import torchvision as tv
import PIL as pil
import matplotlib.pyplot as plt

**Img Loading**

In [0]:
normalization = .5, .5, .5

def load_image(path, size):

  transforms = tv.transforms.Compose([
    tv.transforms.Resize(size),
    tv.transforms.ToTensor(),
    tv.transforms.Lambda(lambda img: img.cuda()),
    tv.transforms.Normalize(normalization, normalization) 
  ])
  return transforms(pil.Image.open(path).convert('RGB'))

def load_images(size, path_content, path_style):
  content = load_image(path_content, size)
  style = load_image(path_style, size)
  result = content.clone().requires_grad_(True)
  return content, style, result

def to_numpy(x):
  return x.cpu().detach().numpy()

def img_to_numpy(img):
  img = np.transpose(to_numpy(img), (1, 2, 0))
  return img * normalization + normalization
  
def plot_img(img):
  plt.imshow(img_to_numpy(img))
  plt.show()
  
def save_img(img, path):
  plt.imsave(path, img_to_numpy(img))

**Define Model**

In [0]:
model = tv.models.vgg19(pretrained = True).cuda()
for param in model.parameters():
  param.requires_grad = False

In [0]:
style_layers = {0, 5, 10, 19, 28}
content_layers = {21}

**Load Features**

In [0]:
def get_feats(img):
  img = img.unsqueeze(0)
  feats_style, feats_content = {}, {}
  for i in range(0, max(style_layers) + 1):
    img = model.features[i](img)
    if i in style_layers:
      feats_style[i] = img.squeeze(0)
    elif i in content_layers:
      feats_content[i] = img.squeeze(0)
  return feats_content, feats_style

def load_feats_content_and_style(content, style):
    feats_content = get_feats(content)[0]
    feats_style = get_feats(style)[1]
    return feats_content, feats_style

def gram(x):
  x = x.view(x.shape[0], -1)
  return t.matmul(x, x.t())

def gram_style(style_feats):
  return { k: gram(style_feats[k]) for k in style_feats }

def get_content_and_gram_style(content, style):
  feats_content, feats_style = load_feats_content_and_style(content, style)
  style_grams = gram_style(feats_style)
  return feats_content, style_grams

In [0]:
path_content = '/content/drive/My Drive/city.jpg'
path_style = '/content/drive/My Drive/starry_night.jpg'
out_img_path = 'City starry night.png'
steps = 101
size = 128

In [0]:
def style_transfer(path_content, path_style, out_img_path, size, steps):
  content, style, result = load_images(size, path_content, path_style)  
  feats_content, style_grams = get_content_and_gram_style(content, style)
  optimizer = t.optim.Adam([result], 1e-3)
  loss_fn = t.nn.MSELoss()

  for i in range(steps):
    feats_result_content, feats_result_style = get_feats(result)
    result_gram_style = gram_style(feats_result_style)
    loss = t.tensor(0.).cuda().requires_grad_(True)
    for layer in result_gram_style:
      loss = loss + loss_fn(result_gram_style[layer], style_grams[layer])
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    if i % 100 == 0:
      print('Step', i + 1)
      plot_img(result)
  print(out_img_path)
  save_img(result, out_img_path)  

In [0]:
style_transfer(path_content, path_style, out_img_path, size, 3000)

style_transfer(out_img_path, path_style, out_img_path, size * 2, 1000)

style_transfer(out_img_path, path_style, out_img_path, size * 4, 1000)

style_transfer(out_img_path, path_style, out_img_path, size * 8, 1000)